# Using CDP File Stores

Methods for retrieving open access data.

### Connecting to the file store

CDP Seattle uses Google Cloud Storage, specifically a storage bucket tied to our database to store our files. However, a properly setup file storage host and associated file store module _should_ have the same functionality.

Here is how to connect to the Seattle file store for **read only** operations.

**Note:** This notebook connects to the staging instance of Seattle's GCS storage. To use production data connect to `cdp-seattle.appspot.com`.

In [1]:
from cdptools.file_stores.gcs_file_store import GCSFileStore

fs = GCSFileStore("stg-cdp-seattle.appspot.com")
fs

<GCSFileStore [stg-cdp-seattle.appspot.com]>

### Getting file URIs

If you know which file you are looking for, simply pass the filename. Files are tagged with a `SHA256` hash of the video uri used to create all downstream artifacts. You can either recreate the hash for the event you are looking for, or query the database for file linkage. For details on database usage, refer to the notebook example on database basics [here](./database.ipynb).

In [2]:
# Using a hash
import hashlib
key = hashlib.sha256("http://video.seattle.gov:8080/media/council/spu_081214V.mp4".encode("utf8")).hexdigest()
fs.get_file_uri(f"{key}_audio.wav")

'https://storage.googleapis.com/stg-cdp-seattle.appspot.com/0292eaf436dc30f91c474ef2ac19e46f715d53f02fc5113c9e56b36be6e13f64_audio.wav'

In [3]:
# Using the database
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase
import pandas as pd

db = CloudFirestoreDatabase("stg-cdp-seattle")
files = pd.DataFrame(db.select_rows_as_list("file"))
fs.get_file_uri(files.loc[0]["filename"])

'https://storage.googleapis.com/stg-cdp-seattle.appspot.com/57848045b7f7b12cf9f38d38e59262233ae386592dd86b47dc65b2bc48142a1d_ts_sentences_transcript_0.txt'

### Downloading files

Unless you want to stream the data from the URI returned, it is recommended to download the file locally prior to usage.

**Note:** You can optionally provide a `save_path` parameter if you don't want the file to be stored with the same name as it is stored with in the bucket.

In [4]:
# Download to local
save_path = fs.download_file(files.loc[files["file_id"] == "fbbafda3-4256-4f41-9383-89ad4ad6b088"]["uri"].values[0])
save_path

PosixPath('/Users/maxfield/Desktop/active/cdp/cdptools/examples/41960b32acc7f79d6bbe41366fb7fac495b81246378bdfd765c37eb27f8c3fcd_ts_sentences_transcript_0.txt')

In [5]:
# Read the transcript
import json
with open(save_path, "r") as read_in:
    sentences = json.load(read_in)
    for s in sentences[:3]:
        print(s)

{'sentence': 'Good morning.', 'start_time': 17.1, 'end_time': 18.1}
{'sentence': "This is a meeting of the city council's education and governance committee.", 'start_time': 18.1, 'end_time': 21.4}
{'sentence': 'Welcome to all of you.', 'start_time': 21.4, 'end_time': 22.6}
